## Haar Cascades

- The "old school" but efficient way of doing object detection.
- Specifically famous for face detection.
- While deep learning (CNNs) has taken over for accuracy, Haar Cascades are still used today in systems with low processing power (like webcams or microcontrollers).
- based on **Viola-Jones algorithm**

### Calculating Haar Features
A Haar feature is essentially calculations that are performed on adjacent rectangular regions at a specific location in a detection window. 

The algorithm uses three types of features: <br><br>
<img src="https://miro.medium.com/v2/resize:fit:684/format:webp/1*yl-BqUzycbyfhPAzwWOddQ.png" width="250">

The sum of the pixels in the white area are substracted from the sum of the pixels in the black area. 

### The Integral Image (or Summed Area Table)
Summing up pixel values is the most computationally expensive part. The Integral Image makes it a lot faster. At any point $(x, y)$, the value in the integral image is the sum of all pixels to the left and above it.

<img src="https://miro.medium.com/v2/resize:fit:864/format:webp/1*dy_lV_6Ne8KSeOWoWB1kAw.png" width="250">

- **The Result**: To calculate the sum of any rectangle, one only needs to look at four corner points in the table: $D - B - C + A$.
- **Complexity**: This takes $O(1)$ time. This means the computer can calculate the "score" for a tiny eye-feature just as fast as a giant torso-feature.

### Adaboost (Adaptive boosting) training - picking the best features
Adaboost essentially chooses the best features and trains the classifiers to use them. It uses a combination of “weak classifiers” to create a “strong classifier” that the algorithm can use to detect objects.

- It evaluates every single feature on a training set of positive (faces) and negative (non-faces) images. 
- It picks the feature that has the lowest error rate. 
- It repeats this, but gives more "weight" to the images that the previous features got wrong. 
- In the end, it narrows down features to a few hundred that actually matter.

### Cascading classifiers
Even with a few hundred "Strong" features, it's too much work to check every pixel of a 4K video. The Cascade is a degenerate tree:
- Stage 0: Checks only 2 features (e.g., "Is there a dark horizontal line across the eyes?").
- Reject: If No, it kills the window immediately (happens for ~50% of the image).
- Stage 1: Checks 10 more features.
- Stage 20: Checks the most complex features.

### Disadvantages
- **Rotation Invariance**: Haar Cascades are very bad at detecting tilted or upside-down faces. They expect the eyes to be horizontal.
- **Occlusion**: If you cover one eye or half the face, the "Cascade" often fails early because it can't find the primary features.
- **Lighting**: They are highly sensitive to lighting because they rely on simple intensity differences. A shadow across half your face can break the detection.
- **False Positives**: In complex backgrounds (like a forest), random shadows can accidentally trigger enough features to "look" like a face to the cascade.

In [ ]:
import cv2

# 1. Load the pre-trained Haar Cascade file
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 2. Load an image and convert to Grayscale (Haar works on light/dark, so color isn't needed)
img = cv2.imread('photo.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 3. Detect faces
# scaleFactor: how much the image size is reduced at each image scale
# minNeighbors: how many 'votes' a region needs to be called a face
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

# 4. Draw rectangles around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

cv2.imshow('Face Detection', img)
cv2.waitKey(0)